## Run Seeded Campaigns

In [3]:
import random
import torch
import numpy as np
from tqdm.auto import trange

from botorch.models import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.fit import fit_gpytorch_model
from botorch.optim import optimize_acqf
from botorch.acquisition import UpperConfidenceBound, ExpectedImprovement
from botorch.test_functions import Hartmann

from botorch.models.transforms import Standardize
from botorch.models.transforms.input import Normalize

from utils import *

from matplotlib import pyplot as plt
plt.style.use('ggplot')

torch.set_default_dtype(torch.float64)

ModuleNotFoundError: No module named 'utils'

In [2]:
def set_seeds(seed : int = 42):
    """set all library random seeds"""
    random.seed(int(seed))
    np.random.seed(int(seed))
    torch.manual_seed(int(seed))

def random_sample(num_points, dims, seed=42):
    """generate random points in the domain"""
    set_seeds(seed)
    return torch.rand(num_points, dims)

# seeds = np.loadtxt('seeds.txt', delimiter=',', dtype=int)

FileNotFoundError: seeds.txt not found.

In [ ]:
seeds = np.loadtxt('seeds.txt', delimiter=',', dtype=int)
n_init = 5
n_campaigns = 5
budget = 30-n_init

ensemble_y = np.zeros((n_campaigns, n_init+budget))
ensemble_X = np.zeros((n_campaigns, 6, n_init+budget))
ensemble_params = np.zeros((n_campaigns, 8, budget))
ensemble_acqf = np.zeros((n_campaigns, budget))

# generate starting data values
HART = Hartmann(dim=6, bounds=[(0,1)]*6 ,negate=True)
X_ = random_sample(n_init, 6)
y_ = torch.tensor([HART(x) for x in X_])[:,None]

# iterate through each campaign (random seed)
for i in (pbar0 := trange(n_campaigns, leave=True, colour="red")):
    set_seeds(seeds[i]) # set the random seed for the campaign
    X = X_.clone()
    y = y_.clone()

    # iterate through each trial up to budget - n_initial
    for j in (pbar1 := trange(budget, leave=True)):

        # build the gp model
        gp = SingleTaskGP(
            train_X = X,
            train_Y = y,
            input_transform=Normalize(d=X.shape[-1]), # normalize X values
            outcome_transform=Standardize(m=y.shape[-1]) # standardize y values
        )

        # fit the model by maximizing the log marginal likelihood
        mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
        mll = fit_gpytorch_model(mll)

        # store the fitted model hyperparameters
        ensemble_params[i,:,j] = np.array([
            *gp.covar_module.base_kernel.lengthscale[0].detach().numpy(),
            gp.covar_module.outputscale.item(),
            gp.likelihood.noise.item()
        ])

        bounds = torch.tensor([[0.0]*X.shape[-1], [1.0]*X.shape[-1]])

        X_new, acq_value = optimize_acqf(
            acq_function= UpperConfidenceBound(gp, beta=2),
            bounds=bounds,
            q=1, # how many new points to generate
            num_restarts = 20, # how many times to restart the optimizer
            raw_samples = 200 # how many initial points to sample acqf space from
        )

        ensemble_acqf[i,j] = acq_value.item()

        X = torch.cat([X, X_new])
        y = torch.cat([y, HART(X=X_new)[:,None]])

        pbar0.set_description(f"Processing Trial {j+1} of Campaign {i+1}")
        pbar1.set_description(f"Campaign {i+1} | Best Value {round(torch.max(y).item(),3)}")

    ensemble_y[i,:] = y.flatten()
    ensemble_X[i,:,:] = X.T